#### Trabajo Práctico N° 1
##### Resolución de sistemas lineales con descomposición LU
Facultad de Ciencias Exactas y Naturales

Álgebra Lineal Computacional 2°C 2023

Mario Sigal Aguirre ~ 157/22 | Luca Jaichenco ~ 591/22

In [37]:
# Imports
import numpy as np
import scipy
import time
import matplotlib.pyplot as plt
np.random.seed(1)

##### Creamos la funcion definir_matiz que genera una matiz n*n de numeros entre -1 y 1

In [38]:
def definir_matriz(shape):
    return np.random.uniform(-1,1, size=shape)

#### Ejercicio 1: descompLU
Dado $ A \in \mathbb{R}^{n \times n}$, la funcion __descompLU__ calcula la descomposicion $LU$ de $A$. 

Se utiliza un algoritmo que realiza la descomposicion por bloques de forma recursiva, teniendo en cuenta el siguiente planteamiento:
$$
A = LU
$$
$$
\begin{pmatrix}
        a_{11} & A_{12} \\
        A_{21} & A_{22}
    \end{pmatrix}
=
\begin{pmatrix}
        1 & 0  \\
        L_{21} & L_{22} 
    \end{pmatrix}
\cdot
\begin{pmatrix}
        u_{11} & U_{12}  \\
        0 & U_{22}
    \end{pmatrix}
$$

$$
\begin{pmatrix}
        a_{11} & A_{12} \\
        A_{21} & A_{22}
    \end{pmatrix}
=
\begin{pmatrix}
        u_{11} & U_{12}  \\
        L_{21}u_{11} & L_{21}U_{12} + L_{22}U_{22} 
    \end{pmatrix}
$$
Se plantean las siguientes ecuaciones para obtener el contenido de los bloques.

$$
u_{11} = a_{11}\\

U_{12} = A_{12}\\

L_{21} = \frac{A_{21}}{u_{11}}\\

L_{22}U_{22} = A_{22} - L_{21}U_{12}
$$

Para obtener $L_{22}$ y $U_{22}$ se calcula recursivamente la descomposicion $LU$ del producto $L_{22}U_{22}$.


El Algoritmo funciona siempre y cuando primer elemento de la diagonal de $A$ sea distinto de cero en cada paso de la recurcion.
$$ a_{11} \neq 0 $$

En cazo contrario, la funcion devuelve a la matriz identidad $I_n$ y la matriz $A$ original.




In [30]:
def descompLU(A):
    n = A.shape[0]
    
    #Caso Base/Matriz de 1x1
    if n == 1:
        if A[0, 0] == 0:
            print("Error: La Matriz tiene un 0 en la diagonal.")
            return np.eye(1), A  # Devuelve la matriz identidad y A original
        else:
            return np.array([[1]]), A  # Matriz L es 1x1 con valor 1 y U es A original

    #Definiciones de bloques:
         
    a11 = A[0, 0] # Obs: Python guarda este valor como un float  
    A12 = A[0, 1:]
    A21 = A[1:, 0] # Obs: Python guarda este vector en formato horizontal
    A22 = A[1:, 1:]

    #Definiciones de matrices complementarias: 
    
    I = np.eye(n)  # Matriz identidad de nxn
    I22 = np.eye(n-1) # Matriz identidad menor
    L = I
    U = np.zeros((n, n))  # Matriz de ceros de nxn
    
    
    # Calculos:
    
    # Calculamos u11 y U12
    if a11 == 0:
        print("Error: La Matriz tiene un 0 en la diagonal.")
        return L, A  # Devuelve la matriz identidad y A original
    
    u11 = a11
    U12 = A12
    
    # Calculamos L21
    L21 = A21 / u11
    
    
    # Calculamos L22 y U22 recursivamente
    L22U22 = A22 - L21.reshape(-1,1)@U12.reshape(1,-1)
    L22, U22 = descompLU(L22U22)
   
    #Verificaciones:
    
    # Verifica si debe devlver (I_n, A) o (L, U)
    # Dividimos los casos en si n == 2 y si n>2, pues en las matrices de 2x2, el bloque U22 siempre es igual a L22U22 y L22 es igual a la identiodad, el problema esta en que cuando n == 2 a su vez es necesario que U22 sea igual a 0 para que se interprete como una matriz singular y devuelva la identidad y la A original
    if  n == 2 and np.array_equal(U22, L22U22) and np.array_equal(L22, I22) and U22 == [[0.]]: 
        return L, A  # Devuelve la matriz identidad y A original
    if n>2 and np.array_equal(U22, L22U22) and np.array_equal(L22, I22): 
        return L, A  # Devuelve la matriz identidad y A original
    
    # Actualizaciones:
    
    # Actualizamos las matrices L y U
    L[1:, 0] = L21
    U[0, 0] = u11
    U[0, 1:] = U12

    # Actualizamos las matrices L y U con los bloques calculados recursivamente
    L[1:, 1:] = L22
    U[1:, 1:] = U22
    
    return L, U



#### Ejercicio 3: resolverLU
Dada una matriz $ A \in \mathbb{R}^{n \times n}$ y un vectoir $ b \in \mathbb{R}^{n} $, la funcion __resolverLU__ devuelve la solucion $x$ del sistema $Ax = b$ mediante la descomposicion $LU$ de $A$, resolviendo los sistemas $Ly = b$ y $Ux = y$.

Decidimos implementar la funcion de dos maneras distintas, una estando programanda directamente desde cero y otra usando la libreria __SciPy__. esto nos permitira analizar la eficiencia de ambas posteriormente.


In [91]:
#resolverLU sin SciPy
def resolverU(U,y): #resuelve el sistema de una matriz triangular superior
    n= U.shape[1]
    x=np.zeros(n)
    
    # Resuelve el sistema:
    for i in range(n-1,-1,-1): #range(n-1,-1,-1) es un range planteado de manera inversa, siendo la sucesion: {n-1, n-2, ...., 2, 1}
        suma = 0
        for j in range(i+1,n):
            suma = suma + U[i,j]*x[j]
                    
        x[i] = (y[i]- suma)/U[i,i]
    return x

def resolverL(L,b): #resuelve el sistema de una matriz triangular Inferior
    
    n= L.shape[1]
    y=np.zeros(n)
    
    # Resuelve el sistema:
    for i in range(0,n):
        suma = 0
        for j in range(0,i):
            suma = suma + L[i,j]*y[j]     
               
        y[i] = (b[i]- suma)/L[i,i]
    return y

def resolverLU(A,b):
    # Calcula la descomposicion LU
    L, U = descompLU(A)

    # Resuelve ambos sistemas:
    y = resolverL(L,b)
    x = resolverU(U,y)   
    return x

A = np.array([[1, 2, 0, 1], 
              [4, 3, 2, 5], 
              [2, 1, 0, 4], 
              [3, 2, 1, 0]])
print(resolverLU(A, [1,2,3,4]))

[ 2.44444444 -0.55555556 -2.22222222 -0.33333333]


In [92]:
#resolverLU con SciPy
def resolverLUSP(A, b):
    # Calcula la descomposicion LU
    L, U = descompLU(A)
    
    # Resuelve ambos sistemas:
    y = scipy.linalg.solve_triangular(L,b, lower = True)
    x = scipy.linalg.solve_triangular(U, y)
    return x

A = np.array([[1, 2, 0, 1], 
              [4, 3, 2, 5], 
              [2, 1, 0, 4], 
              [3, 2, 1, 0]])
print(resolverLU(A, [1,2,3,4]))

[ 2.44444444 -0.55555556 -2.22222222 -0.33333333]


#### Ejercicio 4: Probar resolverLU y calcular el Error Relativo en Norma-2

La funcion __error_rel_n2__ calcula el Error Relatico en Norma-2 entre dos vectores usando funciones de la libreria __NumPy__.

In [43]:
def error_rel_n2(x, y):
    res = np.linalg.norm(x-y,ord=2)/np.linalg.norm(y,ord=2)
    return res

Definimos una matriz $A \in \mathbb{R}^{10 \times 10}$  y un vector $b \in \mathbb{R}^{10}$, ambos de numeros aleatorios en [-1,1),
para calcular el error relativo de norma-2:
$$ \frac{||Ax-b||_2}{||b||_2}.

In [67]:
# Definimos la matriz A y el vector b:
A = definir_matriz((10,10))
b = definir_matriz((1,10))[0]

# Calculamos la solcion del sistema
x = resolverLU(A, b)

# Calculamos el error relativo en norma-2 y su logaritmo natural 
error = error_rel_n2(A@x, b)
logerror = np.log(error_rel_n2(A@x, b))

print("El error relativo entre \nAx = {} \ny b = {}\nes: \n{}.\n\nEl error expresado en escala logaritmica es:\n{} ".format(A@x, b, error, logerror))


[ 0.34899769  0.10293767 -0.35670371  0.56040467 -0.70842914 -0.51023872
 -0.41537248 -0.60813611 -0.82412263  0.73995236]
El error relativo entre 
Ax = [ 0.34899769  0.10293767 -0.35670371  0.56040467 -0.70842914 -0.51023872
 -0.41537248 -0.60813611 -0.82412263  0.73995236] 
y b = [ 0.34899769  0.10293767 -0.35670371  0.56040467 -0.70842914 -0.51023872
 -0.41537248 -0.60813611 -0.82412263  0.73995236]
es: 
2.0696266901777334e-15.

El error expresado en escala logaritmica es:
-33.81140814680645 


#### Ejercicio 5: Cálculo de la Inversa

La funcion __inversa__ recibe una matriz $A \in \mathbb{R}^{n \times n}$ inversible y calcula su inversa por medio de su descomposicion $LU$ y la resolucion de los sistemas $Ly = b$ y $Ux = y$, siendo y los vectores de la base canónica $e_i, 1 \leq i \leq n $ .

Al igual que en el ejercicio 3 Decidimos implementar la funcion de dos maneras distintas, una estando programanda directamente desde cero y otra usando la libreria __SciPy__. esto nos permitira analizar la eficiencia de ambas posteriormente.

In [120]:
#inversa sin SciPy
#copiamos de vuelta resolverU y resolverL para que las celdas del ejercicio 3 y ejercicio 5 se puedan correr independientemente una de otra
def resolverU(U,y): #resuelve el sistema de una matriz triangular superior
    n = U.shape[1]
    x = np.zeros(n)
    # Resuelve el sistema:
    for i in range(n-1,-1,-1): #range(n-1,-1,-1) es un range planteado de manera inversa, siendo la sucesion: {n-1, n-2, ...., 2, 1}
        suma = 0
        for j in range(i+1,n):
            suma = suma + U[i,j]*x[j]
                    
        x[i] = (y[i]- suma)/U[i,i]
    return x

def resolverL(L,b): #resuelve el sistema de una matriz triangular Inferior
    
    n = L.shape[1]
    y=np.zeros(n)
    
    # Resuelve el sistema:
    for i in range(0,n):
        suma = 0
        for j in range(0,i):
            suma = suma + L[i,j]*y[j]     
               
        y[i] = (b[i]- suma)/L[i,i]
    return y


def inversa(A):
    
    # Calcula la descomposicion LU :
    L, U = descompLU(A)
    
    # Se Definen otras matrices a utilizar:
    n = A.shape[0]
    I = np.eye(n)
    Inv = np.zeros((n,n))
    
    # para cada vector columna de la matriz identidad: 
    for i in range(n):
        e_i = I[0:, i]
        
        #Resuelve los sistemas:
        y = resolverL(L,e_i)
        x = resolverU(U,y)
        
        #incorpora los vectores obtenidos a la matriz inversa:
        Inv[0:, i]=x
        
    return Inv



array([[-0.5       ,  1.        , -0.5       ],
       [-0.5       , -2.        ,  1.5       ],
       [ 0.83333333,  1.        , -0.83333333]])

In [121]:
#inversa con SciPy
def inversa(A):
    L, U = descompLU(A)
    In = np.eye(A.shape[0])
    y = scipy.linalg.solve_triangular(L, In, lower = True)
    x = scipy.linalg.solve_triangular(U, y)
    return x


array([[-0.5       ,  1.        , -0.5       ],
       [-0.5       , -2.        ,  1.5       ],
       [ 0.83333333,  1.        , -0.83333333]])

In [54]:

A2 = np.array([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9]])

L2, U2 = descompLU(A2)

print(L2)
print(U2)

A = np.array([[1, 2, 0, 1], 
              [4, 3, 2, 5], 
              [2, 1, 0, 4], 
              [3, 2, 1, 0]])

B = np.array([[1, 2, 3],
              [2, 4, 6],
              [3, 6, 9]])

A_res1 = np.array([[1.,  0. , 0. , 0. ],
                   [4. , 1.,  0. , 0. ],
                   [2. , 0.6 ,1. , 0. ],
                   [3.,  0.8, 0.5 ,1. ]])

A_res2 =  np.array([[ 1. ,  2.  , 0. ,  1. ],
                    [ 0. , -5. ,  2.,  1. ],
                    [ 0. ,  0.,  -1.2 , 1.4],
                    [ 0. ,  0.,   0. , -4.]])
L, U = descompLU(A)

print(L)
print(U)

Error: La Matriz tiene un 0 en la diagonal.
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1.  0.  0.  0. ]
 [4.  1.  0.  0. ]
 [2.  0.6 1.  0. ]
 [3.  0.8 0.5 1. ]]
[[ 1.   2.   0.   1. ]
 [ 0.  -5.   2.   1. ]
 [ 0.   0.  -1.2  1.4]
 [ 0.   0.   0.  -4.5]]


In [34]:
#tests
import unittest
class TestSuite(unittest.TestCase):
    def test_caso_1(self):
        res1, res2 = descompLU(A)
        self.assertEqual(res1, A_res1)
        self.assertEqual(res2, A_res2)
